In [ ]:
import casmnb # notebook setup (local module only) 
tmp_dir = casmnb.notebook_dir() / "bset_example"
input_dir = casmnb.input_dir()
casmnb.autoconfigure()

In [ ]:
import glob, os, pathlib, shutil
from casm.project import Project

**List example prim files**

In [ ]:
# For tutorial notebooks only:
casmnb.list_example_prim()

**Create a project from an example prim**

In [ ]:
## Choose an example prim:
prim_name = 'FCC_Hstrain_prim.json'

## Create a project using the selected prim, 
## in a temporary directory                 
project_path = tmp_dir / "Proj"

if project_path.exists():
    print("Remove existing project...")
    print(project_path)
    shutil.rmtree(project_path)
    print()

print("Create new project directory...")
print(project_path)
project_path.mkdir(parents=True, exist_ok=True)
print()

print("Copy prim:")
src = input_dir / "prim" / prim_name
dest = project_path / "prim.json"
print("src:", src)
print("dest:", dest)
shutil.copyfile(src, dest)
print()

print("Change working directory to:")
print(project_path)
os.chdir(project_path)
print()

project = Project.init(path=project_path)

**Construct a basis set**

In [ ]:
## Get or create a basis set named "chebychev.1"
##
## Note: this id must consist of:
## letters, numbers, underscore only
bset_id = "default"
bset = project.bset.get(bset_id)

## Select the occupation site basis functions
## 
## Note: If the prim doesn't have occupation DoF,
##       then set this to None
## Options: "chebychev", "occupation", None
occ_site_basis_functions_specs = None

## Set the maximum polynomial order
##
## Note: If the prim has continuous DoF, 
##       then make sure this is not None
## Type: int, None
global_max_poly_order = 4

## Add basis set parameters ##
bset.make_bspecs(
    max_length=[0.0, 0.0, 10.01, 7.27, 4.0],
    occ_site_basis_functions_specs=occ_site_basis_functions_specs,
    global_max_poly_order=global_max_poly_order,
)

## Save bspecs.json ##
bset.commit()

## Construct the cluster functions,
## write and compile the Clexulator 
bset.update()

**Enumerate configurations to test correlations**

In [ ]:
enum_id = "occ_by_supercell.1"
enum = project.enum.get(enum_id)
enum.occ_by_supercell(max=4)
enum.commit()

**Calculate correlations**

In [ ]:
corr_calculator = bset.corr_calculator()
corr = corr_calculator.per_unitcell(enum.configuration_set)
print("corr:")
print(corr)
print("shape:", corr.shape)

**Print orbits**

In [ ]:
bset.print_orbits()

**Print functions**

In [ ]:
bset.print_functions()

In [ ]:
for file in bset.out.generated_files.get("all"):
    print(file)

In [ ]:
bset.display_functions(
    id=bset_id,
    linear_orbit_indices=None,  # use i.e. set(range(1,5)) to display a subset
)

In [ ]:
from IPython.display import display, HTML
import mdtex2html

s = R"""# Example-Title

TeX-Formula: 

Occupation site basis functions:

Sublattice 1:
$$
    \begin{array}[rcrrrl]
      & & &\mathrm{Si}, & \mathrm{Ge} &  \\
    \phi_{0,0} &= &[&1, & 1 &] \\
    \phi_{0,1} &= &[&-1, & 1 &] \\
    \end{array} 
$$

$~$

Sublattice 2:
$$
    \begin{array}[rcrrrl]
      & & &\mathrm{Si}, & \mathrm{Ge} &  \\
    \phi_{0,0} &= &[&1, & 1 &] \\
    \phi_{0,1} &= &[&-1, & 1 &] \\
    \end{array} 
$$

- This
- is
    - a List with `inline-Code`
"""
display(HTML(mdtex2html.convert(s)))